In [86]:
import numpy as np
from tqdm import tqdm
from hw2_code.kmeans import KMeans

data = np.random.randn(5, 4)
data[:, 1] = np.array((1,0.5,1,-1,0.641))
# data = np.array([[0.10782656, 0.04982049, 0.78844897, 0.05390398],
#                 [0.16080479, 0.70138883, 0.05805256, 0.07975382],
#                 [0.39637071, 0.23624673, 0.0996817 , 0.26770086],
#                 [0.21741805, 0.56913777, 0.05890126, 0.15454293],
#                 [0.27040961, 0.54778227, 0.01886611, 0.16294201]])

print(data)

[[ 1.82366458  1.         -1.42115534  0.73791448]
 [ 0.20193604  0.5         0.21541547 -1.23314324]
 [ 1.09646068  1.          0.93855823  1.20121923]
 [ 0.13167614 -1.         -0.21449997  0.61133227]
 [ 0.03488984  0.641       1.36107806 -0.60164955]]


In [87]:
### Helper functions
"""
logit is N x D
"""
logit = data

## softmax
def softmax(logit):
    prob = np.exp(logit - np.max(logit)) / np.sum(np.exp(logit - np.max(logit)), axis=-1, keepdims=True)
    return prob

softmax(logit)

## logsumexp
def logsumexp(logit):
    s = np.sum(np.exp(logit + np.max(logit)), axis=-1, keepdims=True)
    return s

logsumexp(logit)

## multinormalPDF
"""
Args:
    points: N x D numpy array
    mu_i: (D,) numpy array, the center for the ith gaussian.
    sigma_i: DxD numpy array, the covariance matrix of the ith gaussian.
Return:
    normal_pdf: (N,) numpy array, the probability density value of N data for the ith gaussian

Hint:
    1. np.linalg.det() and np.linalg.inv() should be handy.
    2. The value in self.D may be outdated and not correspond to the current dataset,
    try using another method involving the current arguments to get the value of D
"""
SIGMA_CONST = 1e-6
mu_i = data.mean(axis=0)
sigma_i = np.cov(data, rowvar=False)
points = data

def multinormalPDF(points, mu_i, sigma_i):
    D = mu_i.shape[0]
    try:
        inv = np.linalg.inv(sigma_i)
    except:
        inv = np.linalg.inv(sigma_i + SIGMA_CONST)

    semi = (points - mu_i) @ inv
    NN =  np.sum((-0.5 * (semi @ (semi.T * (points - mu_i).T))), axis=0)
    normal_pdf = (1/((2*np.pi) ** (D/2))) * (np.linalg.det(sigma_i) ** (-0.5)) * np.exp(NN)
    return normal_pdf

# X = np.reshape(np.arange(20), (5,4))
# muX = X.mean(axis=0)
# print(X)
# print("muX", muX)
# X-muX

multinormalPDF(points, mu_i, sigma_i)

array([2.01504784, 2.01504784, 2.01504784, 2.01504784, 2.01504784])